# Welcome to the Synergy Roundtable
created by Dirk Derichsweiler - 05/10/2018<br>
do not hesitate to contact me: dirk.derichsweiler@hpe.com

jupyter Notebook can be found: https://github.com/dderichswei/synergy


#### requirements
Python + hpOneView Python Library (see: https://github.com/HewlettPackard/python-hpOneView/wiki/HPE-OneView-Python-Windows-Setup-Guide)


#### additional information

On your HPE OneView appliance, or online<br>
https://192.168.146.34/help/cic-rest/en/content/index.html#home.html<br>
https://192.168.146.34/api-docs/current/ <br>
http://www.hpe.com/info/oneview/docs <br>
https://developer.hpe.com/

#### python specific
https://hewlettpackard.github.io/python-hpOneView/index.html<br>
https://github.com/HewlettPackard/python-hpOneView<br>


### Login

<img src="Pictures/Login.gif" height="500" width="500" align="left">

##### import the python OneView library

In [1]:
from hpOneView.oneview_client import OneViewClient

##### configure your environment:

In [2]:
config = {
    "api_version": "500",
    "ip": "10.0.20.50",
    "image_streamer_ip": "10.0.20.54",
    "credentials": {
        "userName": "python",
        "authLoginDomain": "local",
        "password": "python123!"
    }
}

##### login:
create new object oneview_client with the config (see above)

In [3]:
oneview_client = OneViewClient(config)


#### ADVANCED: which functions are available?

In [ ]:
dir(oneview_client)
# dir(oneview_client.storage_volume_templates) # drill down 

### show existing networks

<img src="Pictures/show_network.gif" height="500" width="500" align="left">
<br>

In [ ]:
roundtable_networks=oneview_client.ethernet_networks.get_all()
# print(roundtable_networks)

print("The following networks exists:")
for net in roundtable_networks:
    # print(net['name'] + " VLAN ID:" + str(net['vlanId']))
    print(net['name'] + "    URI: "  + net['uri'] )


### create network

<img src="Pictures/create_network.gif" height="500" width="500" align="left">

In [ ]:
options = {
    "name": "Roundtable - Test Ethernet Network",
    "vlanId": 200,
    "ethernetNetworkType": "Tagged",
    "purpose": "General",
    "smartLink": False,
    "privateNetwork": False,
    "connectionTemplateUri": None
}

ethernet_network = oneview_client.ethernet_networks.create(options)
print("Created ethernet-network " + ethernet_network['name'] + "   URI: " + ethernet_network['uri'] + "successfully.")


### create bulk network
How to set up multiple networks at once.

In [ ]:
options_bulk = {
    "vlanIdRange": "1-5,7,100-109,200",
    "purpose": "General",
    "namePrefix": "Roundtable-Ethernet",
    "smartLink": False,
    "privateNetwork": False,
    "bandwidth": {
        "maximumBandwidth": 10000,
        "typicalBandwidth": 2000
    }
}


ethernet_nets_bulk = oneview_client.ethernet_networks.create_bulk(options_bulk)
for net in ethernet_nets_bulk: 
  print("created network: " + net['name'] + " URI: " + net['uri'])

### delete bulk network
as it´s not needed for the demo

In [ ]:
for net in ethernet_nets_bulk:
    oneview_client.ethernet_networks.delete(net)
    print("delete network: " + net['name'])

### show configured/existing storage (systems and pools)

<img src="Pictures/show_storage.gif" height="500" width="500" align="left">

In [ ]:
storage_system = oneview_client.storage_systems.get_all()
storage_pools = oneview_client.storage_pools.get_all()

# storage_pool_name = 'FC_r1'
# storage_pools = oneview_client.storage_pools.get_by('name', storage_pool_name)[0]

print("Storage Systems:")
for stor in storage_system: 
  print(stor['displayName'] + "    URI: " + stor['uri'])

print("\nStorage Pools:")
for storpool in storage_pools:
  print(storpool['name'] + "    URI: " + stor['uri'])

### show volume templates

In [ ]:
storage_volume = oneview_client.storage_volume_templates.get_all(filter="\"isRoot='False'\"")
for storvol in storage_volume:
    print(storvol['name'])


### create volume template

<img src="Pictures/create_volume_template.gif" height="500" width="500" align="left">

In [ ]:
storage_pool_name = 'FC_r5'

# Get the storage pool by name to use in options
storage_pool = oneview_client.storage_pools.get_by('name', storage_pool_name)[0]

# Gets the first Root Storage Volume Template available to use in options
root_template = oneview_client.storage_volume_templates.get_all(filter="\"isRoot='True'\"")[0]
print(root_template['uri'])

options = {
    "name": "Roundtable Volume Template",
    "description": "",
    "rootTemplateUri": root_template['uri'],
    "properties": {
        "name": {
            "meta": {
                "locked": False
            },
            "type": "string",
            "title": "Volume name",
            "required": True,
            "maxLength": 100,
            "minLength": 1,
            "description": "A volume name between 1 and 100 characters"
        },
        "size": {
            "meta": {
                "locked": False,
                "semanticType": "capacity"
            },
            "type": "integer",
            "title": "Capacity",
            "default": 1073741824,
            "maximum": 17592186044416,
            "minimum": 268435456,
            "required": True,
            "description": "The capacity of the volume in bytes"
        },
        "description": {
            "meta": {
                "locked": False
            },
            "type": "string",
            "title": "Description",
            "default": "",
            "maxLength": 2000,
            "minLength": 0,
            "description": "A description for the volume"
        },
        "isShareable": {
            "meta": {
                "locked": False
            },
            "type": "boolean",
            "title": "Is Shareable",
            "default": False,
            "description": "The shareability of the volume"
        },
        "storagePool": {
            "meta": {
                "locked": False,
                "createOnly": True,
                "semanticType": "device-storage-pool"
            },
            "type": "string",
            "title": "Storage Pool",
            "format": "x-uri-reference",
            "required": True,
            "description": "A common provisioning group URI reference",
            "default": storage_pool['uri']
        },
        "snapshotPool": {
            "meta": {
                "locked": True,
                "semanticType": "device-snapshot-storage-pool"
            },
            "type": "string",
            "title": "Snapshot Pool",
            "format": "x-uri-reference",
            "default": storage_pool['uri'],
            "description": "A URI reference to the common provisioning group used to create snapshots"
        },
        "provisioningType": {
            "enum": [
                "Thin",
                "Full",
                "Thin Deduplication"
            ],
            "meta": {
                "locked": True,
                "createOnly": True
            },
            "type": "string",
            "title": "Provisioning Type",
            "default": "Thin",
            "description": "The provisioning type for the volume"
        }
    }
}

volume_template = oneview_client.storage_volume_templates.create(options)
print("Storage volume created...")

### delete storage template
""" don´t use it, for the demo.. """

In [ ]:
storage_volume = oneview_client.storage_volume_templates.get_all(filter="\"name='Roundtable Volume Template'\"")
for storvol in storage_volume:
    oneview_client.storage_volume_templates.delete(storvol)
    

### show Enclosure / Server Hardware / Bay
<img src="Pictures/show_server_hw.gif" height="500" width="500" align="left">

In [4]:
print ("show enclosure group (enclosure_group_uri)")
enclosure = oneview_client.enclosure_groups.get_all()
for enc in enclosure:
    print(enc['name'] + " " + enc['uri'])

print("\nshow server hardware (server_hardware_type_uri)")
server_hardware_types = oneview_client.server_hardware_types.get_all(sort='name:descending')
# print(server_hardware_types)
for serverhw in server_hardware_types:
    # print(' %s ' % serverhw['model'])
    print(serverhw['model'] + " URI: " + serverhw['uri'] )

print("\nEnclosure/Bay (server_hardware_uri)")   
server_hardware = oneview_client.server_hardware.get_all()
for server in server_hardware:
    print(server['name'] + " " + server['model'] + " " + server['uri'])

show enclosure group (enclosure_group_uri)
SY 3 Frame SAS FC VC /rest/enclosure-groups/73668dd8-d2cf-4004-b7df-8ca7cfeabbc0
SY 1 Frame VC /rest/enclosure-groups/92e527ff-7129-489f-87dc-e7d5821f13c6
SY 3 Frame SAS VC /rest/enclosure-groups/109eed5b-4cff-470e-b2ca-ffb26eb5c678

show server hardware (server_hardware_type_uri)
Synergy 660 Gen9 URI: /rest/server-hardware-types/A435F2DD-9EB9-4F98-B854-1A6B35140DC9
Synergy 660 Gen10 URI: /rest/server-hardware-types/7AA9D8EE-C46A-4901-8D10-C252AF9DB1A4
Synergy 480 Gen9 URI: /rest/server-hardware-types/860CAF19-0D67-401A-A1D2-536AA9356102
Synergy 480 Gen9 URI: /rest/server-hardware-types/1131E4D0-9C51-4615-A20E-CCF00F02A94F
Synergy 480 Gen10 URI: /rest/server-hardware-types/77E175FB-A4AA-4E95-8780-D1E9A40D3887
Synergy 480 Gen10 URI: /rest/server-hardware-types/56C7D5F1-6221-4ACB-938B-21916B7DE5F3

Enclosure/Bay (server_hardware_uri)
CTC Synergy HE01, bay 1 Synergy 480 Gen10 /rest/server-hardware/39313738-3134-5A43-4A37-343030373531
CTC Synergy 

### show Serverprofile


In [6]:
print("show server profile templates:")
all_srv_templates = oneview_client.server_profile_templates.get_all()
for srv_tmp in all_srv_templates:
    print(srv_tmp['name'] + "   URI:   " + srv_tmp['uri'])

print("\nshow server profiles:")
all_profiles = oneview_client.server_profiles.get_all()
for profile in all_profiles:
    print(profile['name']+ "   URI:  " + srv_tmp['uri'])

# my_profile = oneview_client.server_profiles.get_by_name("Roundtable - API Demo Template (DirkD)")

show server profile templates:
SRT-Storage-BfS-W2k16   URI:   /rest/server-profile-templates/1df43d4a-76c3-4821-bff3-729446dc7231
Roundtable - Python (DirkD)   URI:   /rest/server-profile-templates/1e2c29f5-e146-42d4-9f78-a057b41d3953
SRT-LC-WIN-TP   URI:   /rest/server-profile-templates/308e1b0a-2e7c-40aa-bdf9-84eb87b42548
SRT-StorageDemo-ESX   URI:   /rest/server-profile-templates/534d2abd-989c-456e-a84b-8c112ce3fcac
Demo-CTC Windows2016-DC-byI3s   URI:   /rest/server-profile-templates/7a5d77bc-a372-4a44-8a52-b538ba703b97
SRT-Storage-BfS-ESX   URI:   /rest/server-profile-templates/823883ee-0f0e-4554-8e00-a909001b28af
Demo-CTC Windows2016-DC-byI3s_Gen10   URI:   /rest/server-profile-templates/894f818a-9604-48cb-ab9f-ea6c240a9381
DEMO-CTC ESXi Server Template   URI:   /rest/server-profile-templates/ac2995c1-8b21-482b-80e8-7157f1984b75
SRT-LC-WIN BfS   URI:   /rest/server-profile-templates/b18b3cf9-8f47-4a6a-be35-18bba3078acd
DEMO-CTC Stratoscale Template   URI:   /rest/server-profile-t

### create server profile (takes 2-3 minutes)
<br>
<img src="Pictures/boot_synergy.gif" height="500" width="500" align="left">

In [8]:
powerOn = {
    "powerState": "On",
    "powerControl": "MomentaryPress"
}

powerOff = {
    "powerState": "Off",
    "powerControl": "PressAndHold"
}

#template_name = "Roundtable - API Demo Template (DirkD)"
template_name = "Roundtable - Python (DirkD)"
server_name = "Roundtable - API Demo Server (DirkD)"

# server_hardware_uri = '/rest/server-hardware/39313738-3133-5A43-4A37-343030373931' # SY660 G10
server_hardware_uri = '/rest/server-hardware/39313738-3134-5A43-4A37-343030373536' # SY480
server_template_uri = oneview_client.server_profile_templates.get_by_name(template_name)

server_power = oneview_client.server_hardware.update_power_state(powerOff, server_hardware_uri) # turn off server

try:
    print ("create server profile")
    roundtable_server = oneview_client.server_profile_templates.get_new_profile(server_template_uri['uri'])
    roundtable_server["name"] = server_name
    roundtable_server["serverHardwareUri"] = server_hardware_uri

    for name in roundtable_server["osDeploymentSettings"]["osCustomAttributes"]:
        if (name['name'] == "NewUser"):
            name['value']= "dirk"
        if (name['name'] == "NewUserPassword"):
            name['value']= "HalloRoundtable!"   
    profile = oneview_client.server_profiles.create(roundtable_server)

    
except:
    print(server_name + " Server already exists")

server_power = oneview_client.server_hardware.update_power_state(powerOn, server_hardware_uri) # turn on server


create server profile


In [ ]:
#print(roundtable_server["osDeploymentSettings"]["osCustomAttributes"])
#print(roundtable_server)

### Show individual parameters from Image (Imagestreamer)

In [9]:
    #print ("create server profile")
    roundtable_server = oneview_client.server_profile_templates.get_new_profile(server_template_uri['uri'])
    roundtable_server["name"] = server_name
    roundtable_server["serverHardwareUri"] = server_hardware_uri

    for name in roundtable_server["osDeploymentSettings"]["osCustomAttributes"]:
        print(name['name'] + ": " + name['value'])
   


Team0NIC1.networkuri: /rest/ethernet-networks/0a13f077-0b0d-42fc-a693-b4331d36113e
HostName: dirktest
DomainName: demo.local
Team1NIC1.networkuri: /rest/ethernet-networks/a2992999-38b8-40f9-b33f-4646cff31874
NICTeam0Name: team0
Team0NIC1.dhcp: false
Team0NIC2.connectionid: 4


TypeError: can only concatenate str (not "NoneType") to str

### delete server profile
!!! do not use, if not necessary !!!

In [ ]:
server_power = oneview_client.server_hardware.update_power_state(powerOff, server_hardware_uri) # turn on server
oneview_client.server_profiles.delete(profile)

### Excel Export

xlswriter:
https://xlsxwriter.readthedocs.io/

example: we extrace some data to excel XLS

In [ ]:
import xlsxwriter
workbook = xlsxwriter.Workbook('roundtable.xlsx')
worksheet = workbook.add_worksheet()

# Add a bold format to use to highlight cells.
bold = workbook.add_format({'bold': True})

# Text with formatting.
worksheet.write(0,0, 'Synergy Roundtable', bold)

# Start from the first cell below the headers.
row = 4
worksheet.write(row, 0, "Servername", bold)
worksheet.write(row, 1, "Model", bold)
worksheet.write(row, 2, "Memory", bold)
worksheet.write(row, 3, "MAC address", bold)
worksheet.write(row, 4, "WWN address", bold)
worksheet.write(row, 5, "Status", bold)
row += 1

server_hardware = oneview_client.server_hardware.get_all()
#print(server_hardware)

for server in server_hardware:
 col = 0
 for ports in server['portMap']['deviceSlots']:
     for mac in ports['physicalPorts']: 
         for wwnn in mac['virtualPorts']:
             # print(server['name']+ " model:" + server['model'] + " " + str(server['memoryMb']) + " MB mac: " + str(mac['mac']) + " wwnn: " + str(wwnn['wwnn']))
             worksheet.write(row,col, server['name'])
             worksheet.write(row,col+1, server['model'])
             worksheet.write(row,col+2, server['memoryMb'])
             worksheet.write(row,col+3, mac['mac'])
             worksheet.write(row,col+4, wwnn['wwnn'])
             worksheet.write(row,col+5, server['status'])
             row += 1
workbook.close()
print ('Excel File roundtable.xlsx created')

### example: how to get the values from json

In [ ]:
    server_hardware = oneview_client.server_hardware.get_all()
   # print(server_hardware)
    
    for server in server_hardware:
        for ports in server['portMap']['deviceSlots']:
            for mac in ports['physicalPorts']: 
                for wwnn in mac['virtualPorts']:
                    print(server['name']+ " model:" + server['model'] + " " + str(server['memoryMb']) + " MB mac: " + str(mac['mac']) + " wwnn: " + str(wwnn['wwnn'])) 

### application deployment


Webserver NGINX running on docker.

##### What we need:

IP address of deployed system.

<br>
<img src="Pictures/NGINX.png" height="500" width="500" align="left">


In [12]:
import os
import paramiko
import time

ssh = paramiko.SSHClient()

server_name = "Roundtable - API Demo Server (DirkD)"
username = 'root'
password = 'Compaq1!'

deployed_server = oneview_client.server_profiles.get_by_name(server_name)

for name in deployed_server["osDeploymentSettings"]["osCustomAttributes"]:
    if (name['name'] == "Team0NIC1.ipaddress"):
        ip_address = name['value']

        
print("Login with user: " + username + " Server:" + ip_address)        
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())         # add unknown Host-Keys
ssh.connect(ip_address, username=username, password=password)     # login
ssh.exec_command('docker run -d --name nginx -p 80:80 nginx')
#time.sleep(10)
stdin, stdout, stderr = ssh.exec_command("docker exec -it nginx sed -i '\''s/nginx/the Synergy Roundtable/g'\'' /usr/share/nginx/html/index.html", get_pty=True)
print("http://" + ip_address)
# print(stdout.read())
# print(stderr.read())


Login with user: root Server:10.0.20.83
http://10.0.20.83
